# Support Vector Machines (SVM) using Scikit-Learn

**Author**:Sarvin


In [ ]:
#Importing a few essential libraries and then loading one of the data sets.

In [4]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
PLOT = True
DO_DNN = True
DO_SVM = True 
VERBOSE = False
DO_Lin = True
DO_Nu  = True 